# REDES NEURAIS ARTIFICIAIS
## Evaluate Improve e Tunning

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2022-03-05 19:18:24.567012: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-05 19:18:24.567057: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tf.__version__

'2.8.0'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

/usr/lib/python3/dist-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


/usr/lib/python3/dist-packages/sklearn/preprocessing/_encoders.py:351: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = check_array(X, dtype=np.int)
/usr/lib/python3/dist-packages/sklearn/preprocessing/_encoders.py:95: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release 

# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/lib/python3/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/lib/python3/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [9]:
ann = tf.keras.models.Sequential()

2022-03-05 19:19:54.717797: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-05 19:19:54.717818: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-05 19:19:54.717832: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (matheushenrick-PC): /proc/driver/nvidia/version does not exist
2022-03-05 19:19:54.718046: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Adding the input layer and the first hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(units=6, activation='softmax'))

### Adding the second hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=6, activation='softmax'))

### Adding the output layer

In [12]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [13]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [14]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 737us/step - loss: 0.6009 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 666us/step - loss: 0.5328 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 630us/step - loss: 0.5084 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 627us/step - loss: 0.4955 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 600us/step - loss: 0.4807 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 614us/step - loss: 0.4648 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 0s 683us/step - loss: 0.4521 - accuracy: 0.7960
Epoch 8/100
250/250 [==============================] - 0s 675us/step - loss: 0.4431 - accuracy: 0.7960
Epoch 9/100
250/250 [==============================] - 0s 641us/step - loss: 0.4366 - accuracy: 0.7960
Epoch 10/100
250/250 [==============================] - 0s 622us/step - l

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [15]:
y_pred = ann.predict(X_test)
y_pred

array([[0.23748288],
       [0.32494724],
       [0.21008998],
       ...,
       [0.14101449],
       [0.16513255],
       [0.16754436]], dtype=float32)

In [16]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1502   93]
 [ 191  214]]


---
# Avaliando o Modelo

In [18]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [20]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softmax', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softmax'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [21]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

ipykernel_launcher:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:113: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To sile

Epoch 1/10


2022-03-05 19:21:24.222238: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-05 19:21:24.223200: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-05 19:21:24.223930: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (matheushenrick-PC): /proc/driver/nvidia/version does not exist
2022-03-05 19:21:24.225119: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-05 19:21:24.368606: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could

Epoch 1/10
Epoch 1/10


2022-03-05 19:21:24.669596: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-05 19:21:24.669901: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-05 19:21:24.669958: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (matheushenrick-PC): /proc/driver/nvidia/version does not exist
2022-03-05 19:21:24.670802: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-05 19:21:24.704345: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
 27/720 [>.............................] - ETA: 4s - loss: 0.6847 - accuracy: 0.8148   

2022-03-05 19:21:25.906710: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-05 19:21:25.907012: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-05 19:21:25.907099: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (matheushenrick-PC): /proc/driver/nvidia/version does not exist
2022-03-05 19:21:25.907694: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


720/720 [==============================] - 6s 6ms/step - loss: 0.5784 - accuracy: 0.7937
Epoch 2/10
720/720 [==============================] - 7s 6ms/step - loss: 0.5721 - accuracy: 0.7969
Epoch 2/10
720/720 [==============================] - 6s 6ms/step - loss: 0.5752 - accuracy: 0.7926
Epoch 2/10
720/720 [==============================] - 4s 6ms/step - loss: 0.5009 - accuracy: 0.7944
Epoch 3/10
720/720 [==============================] - 5s 6ms/step - loss: 0.5007 - accuracy: 0.7956
Epoch 3/10
720/720 [==============================] - 5s 7ms/step - loss: 0.4977 - accuracy: 0.7975
Epoch 3/10
720/720 [==============================] - 5s 6ms/step - loss: 0.5049 - accuracy: 0.7937
Epoch 3/10
720/720 [==============================] - 5s 7ms/step - loss: 0.4986 - accuracy: 0.7971
Epoch 3/10
720/720 [==============================] - 5s 7ms/step - loss: 0.5014 - accuracy: 0.7962
Epoch 3/10
720/720 [==============================] - 4s 6ms/step - loss: 0.4645 - accuracy: 0.7944
Epoch 4/10


In [22]:
print(mean,variance)

0.8480000019073486 0.014265353873990907


# Melhorando o modelo

In [23]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softmax', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softmax'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [24]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

ipykernel_launcher:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:113: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To sile

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
720/720 [==============================] - 2s 2ms/step - loss: 0.5804 - accuracy: 0.7885
Epoch 2/10
720/720 [==============================] - 2s 2ms/step - loss: 0.5818 - accuracy: 0.7886
Epoch 2/10
720/720 [==============================] - 2s 2ms/step - loss: 0.5805 - accuracy: 0.7890
Epoch 2/10
720/720 [==============================] - 2s 3ms/step - loss: 0.5803 - accuracy: 0.7900
Epoch 2/10
720/720 [==============================] - 2s 3ms/step - loss: 0.5090 - accuracy: 0.7956
Epoch 3/10
720/720 [==============================] - 2s 2ms/step - loss: 0.5079 - accuracy: 0.7969
Epoch 3/10
720/720 [==============================] - 2s 2ms/step - loss: 0.5081 - accuracy: 0.7962
Epoch 3/10
720/720 [==============================] - 2s 3ms/step - loss: 0.5137 - accuracy: 0.7937
Epoch 3/10
720/720 [==============================] - 2s 3ms/step - loss: 0.5074 - accuracy: 0.7967
Epoch 3/10
720/720 [===

In [25]:
print(mean,variance)

0.8107500016689301 0.01867651725106113


# Tuning the ANN

In [26]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softmax', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softmax'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

ipykernel_launcher:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


In [27]:
%%time

grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50


/usr/lib/python3/dist-packages/sklearn/base.py:63: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)
/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:113: DeprecationW

720/720 [==============================] - 1s 594us/step - loss: 0.5741 - accuracy: 0.7971
Epoch 2/50
720/720 [==============================] - 0s 620us/step - loss: 0.5089 - accuracy: 0.7971
Epoch 3/50
720/720 [==============================] - 0s 629us/step - loss: 0.4919 - accuracy: 0.7971
Epoch 4/50
720/720 [==============================] - 0s 579us/step - loss: 0.4674 - accuracy: 0.7971
Epoch 5/50
720/720 [==============================] - 0s 605us/step - loss: 0.4532 - accuracy: 0.7971
Epoch 6/50
720/720 [==============================] - 0s 641us/step - loss: 0.4450 - accuracy: 0.7971
Epoch 7/50
720/720 [==============================] - 0s 610us/step - loss: 0.4417 - accuracy: 0.7971
Epoch 8/50
720/720 [==============================] - 0s 589us/step - loss: 0.4351 - accuracy: 0.7971
Epoch 9/50
720/720 [==============================] - 0s 604us/step - loss: 0.4323 - accuracy: 0.8028
Epoch 10/50
720/720 [==============================] - 0s 590us/step - loss: 0.4265 - accurac

/usr/lib/python3/dist-packages/sklearn/base.py:63: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


720/720 [==============================] - 1s 928us/step - loss: 0.5792 - accuracy: 0.7950
Epoch 2/50
720/720 [==============================] - 1s 1ms/step - loss: 0.5100 - accuracy: 0.7967
Epoch 3/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4900 - accuracy: 0.7967
Epoch 4/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4646 - accuracy: 0.7967
Epoch 5/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4553 - accuracy: 0.7967
Epoch 6/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4509 - accuracy: 0.7967
Epoch 7/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4431 - accuracy: 0.7967
Epoch 8/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4413 - accuracy: 0.7967
Epoch 9/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4335 - accuracy: 0.7976
Epoch 10/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4359 - accuracy: 0.8097
Epoch 11

...

225/225 [==============================] - 1s 1ms/step - loss: 0.6320 - accuracy: 0.7956
Epoch 2/100
225/225 [==============================] - 0s 1ms/step - loss: 0.5556 - accuracy: 0.7957
Epoch 3/100
225/225 [==============================] - 0s 1ms/step - loss: 0.5195 - accuracy: 0.7957
Epoch 4/100
225/225 [==============================] - 0s 1ms/step - loss: 0.5093 - accuracy: 0.7957
Epoch 5/100
225/225 [==============================] - 0s 1ms/step - loss: 0.5079 - accuracy: 0.7957
Epoch 6/100
225/225 [==============================] - 0s 1ms/step - loss: 0.5072 - accuracy: 0.7957
Epoch 7/100
225/225 [==============================] - 0s 1ms/step - loss: 0.5059 - accuracy: 0.7957
Epoch 8/100
225/225 [==============================] - 0s 1ms/step - loss: 0.5049 - accuracy: 0.7957
Epoch 9/100
225/225 [==============================] - 0s 2ms/step - loss: 0.4999 - accuracy: 0.7957
Epoch 10/100
225/225 [==============================] - 0s 2ms/step - loss: 0.4926 - accuracy: 0.7957
E

/usr/lib/python3/dist-packages/sklearn/base.py:63: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


225/225 [==============================] - 1s 1ms/step - loss: 0.6306 - accuracy: 0.7957
Epoch 2/100
225/225 [==============================] - 0s 1ms/step - loss: 0.5549 - accuracy: 0.7961
Epoch 3/100
225/225 [==============================] - 0s 2ms/step - loss: 0.5186 - accuracy: 0.7961
Epoch 4/100
225/225 [==============================] - 0s 2ms/step - loss: 0.5083 - accuracy: 0.7961
Epoch 5/100
225/225 [==============================] - 0s 1ms/step - loss: 0.5066 - accuracy: 0.7961
Epoch 6/100
225/225 [==============================] - 0s 1ms/step - loss: 0.5064 - accuracy: 0.7961
Epoch 7/100
225/225 [==============================] - 0s 1ms/step - loss: 0.5050 - accuracy: 0.7961
Epoch 8/100
225/225 [==============================] - 0s 1ms/step - loss: 0.5029 - accuracy: 0.7961
Epoch 9/100
225/225 [==============================] - 0s 2ms/step - loss: 0.4972 - accuracy: 0.7961
Epoch 10/100
225/225 [==============================] - 0s 1ms/step - loss: 0.4884 - accuracy: 0.7961
E

/usr/lib/python3/dist-packages/sklearn/model_selection/_search.py:821: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
/usr/lib/python3/dist-packages/sklearn/base.py:63: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  new_object = klass(**new_object_params)


320/320 [==============================] - 1s 945us/step - loss: 0.6157 - accuracy: 0.7934
Epoch 2/100
320/320 [==============================] - 0s 1ms/step - loss: 0.5307 - accuracy: 0.7960
Epoch 3/100
320/320 [==============================] - 0s 1ms/step - loss: 0.5093 - accuracy: 0.7960
Epoch 4/100
320/320 [==============================] - 0s 1ms/step - loss: 0.5058 - accuracy: 0.7960
Epoch 5/100
320/320 [==============================] - 0s 1ms/step - loss: 0.5053 - accuracy: 0.7960
Epoch 6/100
320/320 [==============================] - 0s 1ms/step - loss: 0.5011 - accuracy: 0.7960
Epoch 7/100
320/320 [==============================] - 0s 1ms/step - loss: 0.4941 - accuracy: 0.7960
Epoch 8/100
320/320 [==============================] - 0s 1ms/step - loss: 0.4834 - accuracy: 0.7960
Epoch 9/100
320/320 [==============================] - 0s 1ms/step - loss: 0.4702 - accuracy: 0.7960
Epoch 10/100
320/320 [==============================] - 0s 2ms/step - loss: 0.4621 - accuracy: 0.7960

In [28]:
print('Best params:\n',grid_search.best_params_,'\nBest accuracy:\n',grid_search.best_score_)

Best params:
 {'batch_size': 25, 'epochs': 100, 'optimizer': 'rmsprop'} 
Best accuracy:
 0.863125


# Halving Grid Search

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softmax', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softmax'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

halving_cv = HalvingGridSearchCV(estimator = classifier,
                                  param_grid = parameters,
                                  scoring="accuracy", 
                                  n_jobs=-1, 
                                  min_resources="exhaust", 
                                  factor=3,
                                  cv = 10)

In [ ]:
%%time
halving_cv = halving_cv.fit(X_train, y_train)

In [ ]:
print('Best params:\n',halving_cv.best_params_,'\nBest accuracy:\n',halving_cv.best_score_)

Best params:
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'rmsprop'} 
Best accuracy:
 0.8346683354192741


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)